<a href="https://colab.research.google.com/github/hugomrtz/Black-Scholes-Pricer/blob/main/blackscholes_pricer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import scipy.stats as si
import matplotlib.pyplot as plt
import plotly.graph_objects as go
!pip install yfinance
import yfinance as yf

In [2]:
ticker = 'AAPL' # Here an example for Apple data
data = yf.Ticker(ticker)
spot_price = data.history(period='1d')['Close'][0]

S = spot_price   # actual spot
K = 180   # strike
T = 0.5  # time to maturity (in years)
r = 0.045  # risk-free rate
sigma = 0.15 # volatility


def black_scholes_full(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    call = S * si.norm.cdf(d1) - K * np.exp(-r * T) * si.norm.cdf(d2)
    put = K * np.exp(-r * T) * si.norm.cdf(-d2) - S * si.norm.cdf(-d1)

    delta_call = si.norm.cdf(d1)
    delta_put = delta_call - 1

    gamma = si.norm.pdf(d1) / (S * sigma * np.sqrt(T))

    theta_call = (-S * si.norm.pdf(d1) * sigma / (2 * np.sqrt(T)) -
                  r * K * np.exp(-r * T) * si.norm.cdf(d2))
    theta_put = (-S * si.norm.pdf(d1) * sigma / (2 * np.sqrt(T)) +
                 r * K * np.exp(-r * T) * si.norm.cdf(-d2))

    vega = S * si.norm.pdf(d1) * np.sqrt(T)

    return {
        'call_price': call,
        'put_price': put,
        'd1': d1,
        'd2': d2,
        'delta_call': delta_call,
        'delta_put': delta_put,
        'gamma': gamma,
        'theta_call': theta_call,
        'theta_put': theta_put,
        'vega': vega
    }

def plot_option_payoffs(K, option_type='call'):
    S_range = np.linspace(K * 0.5, K * 1.5, 200)
    payoff = []

    for S in S_range:
        if option_type == 'call':
            payoff.append(max(S - K, 0))
        else:
            payoff.append(max(K - S, 0))

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=S_range, y=payoff, mode='lines',
                             name=f"{option_type.capitalize()} Payoff",
                             line=dict(color='royalblue')))

    fig.add_vline(x=K, line=dict(color='red', dash='dash'), annotation_text="Strike (K)", annotation_position="top right")
    fig.update_layout(title=f"{option_type.capitalize()} Option payoff at maturity",
                      xaxis_title='Spot price at maturity (S)',
                      yaxis_title='Payoff',
                      template='plotly_white')
    fig.show()


result = black_scholes_full(S, K, T, r, sigma)
print(f"Current Spot price for {ticker}: {spot_price:.2f}") # Current spot price (at the last closure)

print("Call price:", round(result['call_price'], 2))
print("Put price:", round(result['put_price'], 2))

print(f"d1: {result['d1']:.4f} | d2: {result['d2']:.4f}")

print(f"Delta Call: {result['delta_call']:.4f} | Delta Put: {result['delta_put']:.4f}")
print(f"Gamma: {result['gamma']:.4f}")
print(f"Theta Call: {result['theta_call']:.4f} | Theta Put: {result['theta_put']:.4f}")
print(f"Vega: {result['vega']:.4f}")

# Interactive visualizations payoff
plot_option_payoffs(K, option_type='call')
plot_option_payoffs(K, option_type='put')

<ipython-input-2-2d58f1160b9c>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  spot_price = data.history(period='1d')['Close'][0]


Current Spot price for AAPL: 203.27
Call price: 28.08
Put price: 0.8
d1: 1.4114 | d2: 1.3054
Delta Call: 0.9209 | Delta Put: -0.0791
Gamma: 0.0068
Theta Call: -10.3371 | Theta Put: -2.4173
Vega: 21.1783
